<a href="https://colab.research.google.com/github/LAMezaCaballero/practica_ETL/blob/main/etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
from google.colab import files #para cargar archivos desde local a red
from pydrive2.auth import GoogleAuth # usar el googledrive
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# cargar LOS DATOS

In [26]:
df = pd.read_csv("/content/drive/MyDrive/henry/practicaVinos/winemag-data-130k-v2.csv")
df= pd.DataFrame(df)
df.drop(columns='Unnamed: 0', axis=0, inplace= True) # eliminar la columna indice extra
df.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [33]:
df_0= len(df)
print(df.info(), df.describe(), '\n cantidad de lineas', df_0)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   country                129908 non-null  object 
 1   description            129971 non-null  object 
 2   designation            92506 non-null   object 
 3   points                 129971 non-null  int64  
 4   price                  120975 non-null  float64
 5   province               129908 non-null  object 
 6   region_1               108724 non-null  object 
 7   region_2               50511 non-null   object 
 8   taster_name            103727 non-null  object 
 9   taster_twitter_handle  98758 non-null   object 
 10  title                  129971 non-null  object 
 11  variety                129970 non-null  object 
 12  winery                 129971 non-null  object 
dtypes: float64(1), int64(1), object(11)
memory usage: 12.9+ MB
None               points     

eliminar columnas con la mayor contadidad de NaN

In [35]:
df.drop(columns='region_2', axis=0, inplace= True) # eliminar la columna indregion extra
df.dropna(axis=0,inplace= True, how= 'all', subset='taster_twitter_handle') # elimina la fila que tenga nan en la columna de taster
df.tail()

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
129965,France,"While it's rich, this beautiful dry wine also ...",Seppi Landmann Vallée Noble,90,28.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Rieflé-Landmann 2013 Seppi Landmann Va...,Pinot Gris,Domaine Rieflé-Landmann
129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
129970,France,"Big, rich and off-dry, this is powered by inte...",Lieu-dit Harth Cuvée Caroline,90,21.0,Alsace,Alsace,Roger Voss,@vossroger,Domaine Schoffit 2012 Lieu-dit Harth Cuvée Car...,Gewürztraminer,Domaine Schoffit


In [42]:
df_1= 100-(len(df)/df_0 * 100)
print(df.info(), df.describe(), '\n el porcentaje de lineas borradas', df_1)

<class 'pandas.core.frame.DataFrame'>
Index: 98758 entries, 0 to 129970
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                98695 non-null  object 
 1   description            98758 non-null  object 
 2   designation            71118 non-null  object 
 3   points                 98758 non-null  int64  
 4   price                  91559 non-null  float64
 5   province               98695 non-null  object 
 6   region_1               79859 non-null  object 
 7   taster_name            98758 non-null  object 
 8   taster_twitter_handle  98758 non-null  object 
 9   title                  98758 non-null  object 
 10  variety                98758 non-null  object 
 11  winery                 98758 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 9.8+ MB
None              points         price
count  98758.000000  91559.000000
mean      88.635594     35.2803

In [43]:
df.dropna(axis= 0, inplace=True)
df[df['variety']=='Pinot Noir']

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
25,US,Oak and earth intermingle around robust aromas...,King Ridge Vineyard,87,69.0,California,Sonoma Coast,Virginie Boone,@vboone,Castello di Amorosa 2011 King Ridge Vineyard P...,Pinot Noir,Castello di Amorosa
35,US,As with many of the Erath 2010 vineyard design...,Hyland,86,50.0,Oregon,McMinnville,Paul Gregutt,@paulgwine,Erath 2010 Hyland Pinot Noir (McMinnville),Pinot Noir,Erath
78,US,Some rosés are made simply by bleeding the jui...,Rosé of,86,25.0,Oregon,Eola-Amity Hills,Paul Gregutt,@paulgwine,Z'IVO 2015 Rosé of Pinot Noir (Eola-Amity Hills),Pinot Noir,Z'IVO
145,US,"Cooked cranberry is spiced with anise, pepperc...",Guidotti Vineyard,91,64.0,California,Santa Lucia Highlands,Matt Kettmann,@mattkettmann,Testarossa 2013 Guidotti Vineyard Pinot Noir (...,Pinot Noir,Testarossa
...,...,...,...,...,...,...,...,...,...,...,...,...
129818,France,"Concentrated, with dark tannins to go with the...",Les Damodes,89,115.0,Burgundy,Nuits-St.-Georges,Roger Voss,@vossroger,Louis Latour 2009 Les Damodes (Nuits-St.-Geor...,Pinot Noir,Louis Latour
129862,US,Earthy forest floor and Asian spice ride atop ...,Stephanie's Cuvée,90,25.0,California,Green Valley,Virginie Boone,@vboone,River Road 2014 Stephanie's Cuvée Pinot Noir (...,Pinot Noir,River Road
129872,US,Almost like liquid cherry pie—but not sweet—th...,Slow Lope'n Vineyard,90,32.0,California,Anderson Valley,Jim Gordon,@gordone_cellars,Brutocao 2013 Slow Lope'n Vineyard Pinot Noir ...,Pinot Noir,Brutocao
129912,US,From the Ranch House block of the famous viney...,Durell Vineyard,91,55.0,California,Sonoma Coast,Virginie Boone,@vboone,Dunstan 2014 Durell Vineyard Pinot Noir (Sonom...,Pinot Noir,Dunstan


In [44]:
df_1= 100-(len(df)/df_0 * 100)
print(df.info(), df.describe(), '\n el porcentaje de lineas borradas', df_1)

<class 'pandas.core.frame.DataFrame'>
Index: 52106 entries, 4 to 129970
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                52106 non-null  object 
 1   description            52106 non-null  object 
 2   designation            52106 non-null  object 
 3   points                 52106 non-null  int64  
 4   price                  52106 non-null  float64
 5   province               52106 non-null  object 
 6   region_1               52106 non-null  object 
 7   taster_name            52106 non-null  object 
 8   taster_twitter_handle  52106 non-null  object 
 9   title                  52106 non-null  object 
 10  variety                52106 non-null  object 
 11  winery                 52106 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 5.2+ MB
None              points         price
count  52106.000000  52106.000000
mean      89.031474     39.7448

In [45]:
df.dropna(axis=0,inplace= True, how= 'all')
df.head()

,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
5,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Michael Schachner,@wineschach,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
6,Italy,"Here's a bright, informal red that opens with ...",Belsito,87,16.0,Sicily & Sardinia,Vittoria,Kerin O’Keefe,@kerinokeefe,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,Terre di Giurfo
9,France,This has great depth of flavor with its fresh ...,Les Natures,87,27.0,Alsace,Alsace,Roger Voss,@vossroger,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,Jean-Baptiste Adam
10,US,"Soft, supple plum envelopes an oaky structure ...",Mountain Cuvée,87,19.0,California,Napa Valley,Virginie Boone,@vboone,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,Kirkland Signature


In [46]:
df_1= 100-(len(df)/df_0 * 100)
print(df.info(), df.describe(), '\n el porcentaje de lineas borradas', df_1)

<class 'pandas.core.frame.DataFrame'>
Index: 52106 entries, 4 to 129970
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   country                52106 non-null  object 
 1   description            52106 non-null  object 
 2   designation            52106 non-null  object 
 3   points                 52106 non-null  int64  
 4   price                  52106 non-null  float64
 5   province               52106 non-null  object 
 6   region_1               52106 non-null  object 
 7   taster_name            52106 non-null  object 
 8   taster_twitter_handle  52106 non-null  object 
 9   title                  52106 non-null  object 
 10  variety                52106 non-null  object 
 11  winery                 52106 non-null  object 
dtypes: float64(1), int64(1), object(10)
memory usage: 5.2+ MB
None              points         price
count  52106.000000  52106.000000
mean      89.031474     39.7448

# **CONCLUSION**

Se redujo un 59.9% de los datos por falta de valores en al menos un punto de la fila.